# MATH 405/607 

# Numerical Methods for Differential Equations

[[Instructor: Christoph Ortner]](http://www.math.ubc.ca/~ortner/)  [[course page]](https://github.com/cortner/math405_2022)

# Quadrature 

* interpolatory quadrature
* composite quadrature 
* Newton-Cotes quadrature 

### Literature 

* [Driscoll, Fundamentals of Numerical Computations](https://fncbook.github.io/fnc/localapprox/overview.html)
* Süli and Mayers, An Introduction to Numerical Analysis, Ch 6-11 : range of elementary and advanced material
* Trefethen, Approximation Theory and Approximation Practise : more advanced material

In [ ]:
include("math405.jl")

## Numerical Integration / Quadrature 

**Motivation:** Suppose we wish to numerically evaluate an integral 
$$ 
 I =  \int_a^b f(x) \,dx 
$$

A far-reaching idea is to approximate $f \approx p_N$ with a  polynomial and therefore approximate the integral 
$$ 
 I \approx I_N := \int_{a}^b p_N(x) \, dx
$$ 
Because $p_N$ is a polynomial, $I_N$ can be evaluated analytically. Applying this idea in a "piecewise" fashion (cf splines!) leads to so-called *composite quadrature rules*.

**Topics:** midpoint rule, trapezoidal rule, Simpson rule, Newton-Cotes quadrature, Clenshaw-Curtis quadrature, Gauss quadrature

## Trapezoidal rule 

In [ ]:
f0 = x -> x/(1+10*x^2)

MATH405.illustrate_trapezoidal(f0, 8)

Choose a partition $a = x_0 < x_1 < \dots, x_M = b$ then the *trapezoidal  rule* is 
$$ 
  I_N := \sum_{m = 1}^M \frac{x_m-x_{m-1}}{2} \big(f(x_{m-1}) + f(x_m)\big)
$$

Implementation of the trapezoidal rule:

In [ ]:
trapezoidal(g, N) = ( 0.5/N * (g(0.0) + g(1.0))
                      + sum(g(n/N) for n=1:N-1) / N )

In [ ]:
# test it on our example function 
@show trapezoidal(f0, 5);
@show trapezoidal(f0, 10);
@show trapezoidal(f0, 20);
@show trapezoidal(f0, 40);

### Error analysis

Suppose that $x_m = (b-a) / M$, $m = 0, \dots, M$ and $h = (b-a)/M = x_m - x_{m-1}$, then we know that the error in the linear interpolant is
$$
    \| f - s_1 \|_{\infty} \leq \frac{h^2}{8} \| f'' \|_{\infty} 
$$

and we immediately obtain 
$$
    \bigg| \int_a^b f - s_1 \,dx \bigg| \leq (b-a) \| f - s_1 \|_\infty \leq \frac{ (b-a) h^2}{8} \|f''\|_\infty.
$$

But we can do a little better if we remember that $f - s_1 = \frac{1}{2} (0-x) (h-x) f''(\xi)$. Therefore: 
$$ 
     \bigg|\int_0^h f - s_1 \,dx \bigg| 
     \leq \frac{\|f''\|_\infty}{2} \int_0^h | x (h-x) | \,dx
     \leq \frac{h^3 \|f''\|_\infty}{12}
$$

**Proposition:** The error for the composite trapezoidal rule is bounded by 
$$ 
  |I - I_M| \leq \frac{ (b-a) h^2}{12} \|f''\|_\infty.
$$

In [ ]:
f = x -> 1/(1+10*x^2)
If = atan(sqrt(10)) / sqrt(10) # the exact value of the integral -> CHECK THIS!
maxd2f = 1/sqrt(10) 

trapezoidal(f, N) = ( 0.5/N * (f(0.0) + f(1.0)) + sum(f(n/N) for n=1:N-1)/N )

NN = [4, 8, 16, 32, 64, 128]
If_N = [ trapezoidal(f, N) for N in NN ]

plot(NN, abs.(If_N .- If), lw=2, m=:o, ms=6, label = L"|I - I_N|", 
     xaxis = :log, yaxis = :log, size = (600, 300), 
     title = "Convergence Trapezoidal rule")
plot!(NN[2:5], (maxd2f/12) * NN[2:5].^(-2), lw=2, ls=:dash, c=:black, label =  L"\sim N^{-2}")

### Midpoint rule

Use piecewise constant interpolation at the interval midpoints: Choose a partition $a = x_0 < x_1 < \dots, x_M = b$ then the *mid-point rule* is 
$$ 
  I_N := \sum_{m = 1}^M (x_m-x_{m-1}) f\big( 0.5 (x_{m-1}+x_m) \big)
$$

In [ ]:
MATH405.illustrate_midpoint(f0, 8)

**Challenge:** Guess the order of approximation?
$$ 
  I_M := \sum_{m = 1}^M (x_m-x_{m-1}) f\big( 0.5 (x_{m-1}+x_m) \big)
$$

In [ ]:
midpoint(f, N) = sum( f(x) for x in range(0.5/N, 1-0.5/N, length=N) ) / N

If_mid = [ midpoint(f, N) for N in [4, 8, 16, 32, 64, 128] ]

plot(NN, abs.(If_N .- If), lw=2, m=:o, ms=6, label = "trapezoidal", 
    xaxis = :log, yaxis = :log, size = (600, 300), 
    title = "Convergence Midpoint Rule")
plot!(NN, abs.(If_mid .- If), lw=2, m=:o, ms=6, label = "midpoint")
plot!(NN[2:5], (maxd2f/12)*NN[2:5].^(-2), lw=2, ls=:dash, c=:black, label =  L"\sim N^{-2}")

**Analysis of the mid-point rule:**  Piecewise constant approximation suggests that the quadrature error should be $O(h) = O(N^{-1})$ but we observe $O(h^2) = O(N^{-2})$. To explain this we can make a more careful Taylor expansion: 

$$
  f(x) = f(h/2) + f'(h/2) (x - h/2) + f''(\xi) (x-h/2)^2/2 
$$

$$
\begin{aligned}
  \int_0^h f(x) dx &= h f(h/2) + f'(h/2) \underset{=0}{\underbrace{\int_0^h (x-h/2) \,dx}} + {\rm remainder} \\ 
  |I - I_h| &\leq \frac{\|f''\|_\infty}{2} \int_0^h (x - h/2)^2 dx 
      = \frac{\|f''\|_\infty h^3}{24}
\end{aligned}
$$

Now we can argue as in the analysis of the trapezoidal rule to prove the following result: 

**Proposition:** Let $I_N$ denote the composite mid-point rule, then 
$$ 
   |I - I_N| \leq \frac{(b-a) h^2}{24}  \|f''\|_{\infty}.
$$

### Newton-Cotes Quadrature 

* Take $x_0, \dots, x_N$ equi-spaced, and $p$ the nodal interpolant of $f$, then the *open Newton-Cotes formula is*
$$
  \int_{x_0}^{x_N} f(x)\,dx \approx \int_{x_0}^{x_N} p(x)\,dx
  = (x_N-x_0) \sum_{n = 0}^N w_n f(x_n)
$$
(e.g. trapezoidal rule with $N = 1$

* Take $q$ the nodal interpolant at $x_1, \dots, x_{N-1}$ then the *closed Newton-Cotes formula* is 
$$
  \int_{x_0}^{x_N} f(x)\,dx \approx \int_{x_0}^{x_N} q(x)\,dx
   = (x_N - x_0) \sum_{ n = 1}^{N-1} w_n f(x_n).
$$
(e.g. mid-point rule with $N = 2$)


But remember that interpolation at equi-spaced points is unstable so we should probably not let $N \to \infty$!!! For small or moderate $N$, these can be good quadrature rules but they are rarely (never?) used for higher degrees where much better choices are available! We will explore this in a moment."

### Simpson Rule 

E.g. the closed Newton-Cotes formula with $N=3$ is the Simpson rule 
$$ 
    I = \int_0^h f(x) dx \approx I_h = h  \Big(\frac{1}{6} f(0) + \frac{4}{6} f(h/2) + \frac{1}{6} f(h) \Big)
$$
with error 
$$ 
   |I - I_h| \leq \frac{ \|f^{(iv)} \|_\infty h^5}{2280}
$$

Determine weights: $p = $ quadratic interpolant of $f$ at nodes $0, h/2, h$
$$\begin{aligned}
   I &= \int_0^h f(x) dx \approx \int_0^h p(x) dx =: I_h \\
   p(x) &= f(0) L_0(x) + f(h/2) L_1(x) + f(h) L_2(x) \\ 
   I_h &=  \int_0^h p(x) dx = f(0) \int_0^h L_0(x) dx + f(h/2) \int_0^h L_1(x) dx + \dots
\end{aligned}$$

### General interpolatory quadrature rules

Take general interpolation points $x_0 < \dots, x_N$ and $p(x)$ the nodal interpolant of $f$ and approximate
$$
   I = \int_{x_0}^{x_N} f(x)\,dx \approx I_N := \int_{x_0}^{x_N} p(x) \,dx 
$$
then we have 
$$ 
I_N =  (x_N-x_0) \sum_{n = 0}^N f(x_n)
    \underset{=: w_n}{ \underbrace{ \frac{\int_{x_0}^{x_N} L_n(x) \,dx}{x_N-x_0} } } 
$$


In [ ]:
function unstable_quad_weights(X)
    N = length(X)-1
    Vt = [ x^n for x in X, n = 0:N ]'
    return qr(Vt) \ (1 ./ collect(1:N+1))
end

unstable_quad_weights(range(0, 1, length=10))

In [ ]:
interp_quad(f, X) = dot(f.(X), unstable_quad_weights(X))

newton_cotes_quad(f, N) = interp_quad(f, range(0, 1, length=N+1))

In [ ]:
f = x -> 1 / (1 + 25 * x^2)
If = atan(5) / 5

NN = 4:4:40
If_nc = [ newton_cotes_quad(f, N) for N in NN ]

plot(NN, abs.(If_nc .- If), lw=2, m=:o, ms=6, label = "trapezoidal", 
    yaxis = :log, size = (600, 300), 
    title = "Convergence Newton-Cotes")
plot!(NN[1:4], 0.01*exp.(- 0.4 * NN[1:4]), lw=2, ls=:dash, c=:black, label =  L"\sim e^{- 2N/5}")

**Conclusion:** Our implementation of Newton-Cotes quadrature is unstable. This is entirely consistent with our expectation from higher-order polynomial interpolation with equidistant interpolation points. 

### Clenshaw-Curtis Quadrature 

Remember from the lecture on interpolation that Chebyshev interpolation is far superior to equispaced interpolation - the approximation errors are much smaller. This motivates us to use the Chebyshev nodes also for interpolation. This is called [Clenshaw-Curtis quadrature](https://en.wikipedia.org/wiki/Clenshaw–Curtis_quadrature); see also this beautiful [review paper](http://people.maths.ox.ac.uk/trefethen/CC.pdf)

From the general framework (last slide) the quadrature nodes (= interpolation nodes) already fully specify the quadrature rule, we just need to get the weights. For Clenshaw-Curtis quadrature there is a very nice but intricate way to compute those weights via a Fast Fourier Transform. Here - just for the sake of demonstration - we use the unstable implementation via the Vandermonde matrix. 

In [ ]:
clenshaw_curtis_quad(f, N) = interp_quad(f, 0.5 .+ 0.5 * reverse(cos.( range(0, π, length=N+1) )))


In [ ]:
NN = 4:4:40

If_nc = [ newton_cotes_quad(f, N) for N in NN ]
If_cc = [ clenshaw_curtis_quad(f, N) for N in NN ]

plot(NN, abs.(If_nc .- If), lw=2, m=:o, ms=6, label = "Newton-Cotes", 
    yaxis = :log, size = (600, 300), title = "Convergence Spectral Quadrature")
plot!(NN, abs.(If_cc .- If), lw=2, m=:o, ms=6, label = "Clenshaw-Curtis")
plot!(NN[1:4], 0.01*exp.(- NN[1:4]), lw=2, ls=:dash, c=:black, label =  L"\sim exp(-a N)")

In `math405.jl` there is an [alternative implementation](http://people.maths.ox.ac.uk/trefethen/CC.pdf) that exploits the fast Chebyshev transform. We won't study this in detail, but just take it as an illustration of "what else is out there" and another illustration of the importance of numerically stable algorithms!

In [ ]:
If_scc = [ MATH405.stable_clenshaw_curtis(f, N) for N in NN ]

plot(NN, abs.(If_nc .- If), lw=2, m=:o, ms=6, label = "Newton-Cotes", 
    yaxis = :log, size = (600, 300), title = "Convergence Spectral Quadrature")
plot!(NN, abs.(If_cc .- If), lw=2, m=:o, ms=6, label = "Clenshaw-Curtis")
plot!(NN, abs.(If_scc .- If), lw=2, m=:o, ms=6, label = "stable Clenshaw-Curtis")

plot!(NN[1:5], 0.01*exp.(- NN[1:5]), lw=2, ls=:dash, c=:black, label =  L"\sim N^{-2}")

## Summary Quadrature 

* Approximate $\int f$ by $\int p$ where $p$ is a polynomial interpolant
* increase polynomial degree or decrease mesh-size (splines!) for convergence
* close connection to polynomial and and spline approximation

### Further reading

* Gauss quadrature 
* Adaptive Quadrature
* $hp$ refinement

And some code: 
* [`Cubature.jl`](https://github.com/JuliaMath/Cubature.jl)
* [`QuadratureRules.jl`](https://github.com/JuliaGNI/QuadratureRules.jl)